In [324]:
import pandas as pd
import numpy as np
import matplotlib.animation as ani
from requests import get
import re
from bs4 import BeautifulSoup
import warnings
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap


In [167]:

east_master = pd.DataFrame()
west_master = pd.DataFrame()

for year in range(2000,2021):
    r = get(f'https://www.basketball-reference.com/friv/standings.fcgi?month=5&day=15&year='+str(year))
    if r.status_code==200:
        soup = BeautifulSoup(r.content, 'html.parser')
        e_table = soup.find('table', attrs={'id': 'standings_e'})
        e_df = pd.read_html(str(e_table))[0]
        w_table = soup.find('table', attrs={'id': 'standings_w'})
        w_df = pd.read_html(str(w_table))[0]
        e_df.rename(columns={'Eastern Conference': 'TEAM','W':str(year)}, inplace=True)
        w_df.rename(columns={'Western Conference': 'TEAM','W':str(year)}, inplace=True)
        e_df['TEAM'] = e_df.TEAM.str.replace(r'([^\s\w])+','')
        w_df['TEAM'] = w_df.TEAM.str.replace(r'([^\s\w])+','')
        e_df = e_df.iloc[:, :-7]
        e_df = e_df.iloc[:, :-7]
        if east_master.empty is True:
            east_master = e_df.copy()
        else:
             east_master = pd.merge(e_df, east_master, on='TEAM')
        if east_master.empty is True:
            east_master = e_df.copy()
        else:
             east_master = pd.merge(e_df, east_master, on='TEAM')
        del e_df
        del w_df
        




In [55]:
east_master = pd.DataFrame()
west_master = pd.DataFrame()

for year in range(2000,2021):
    r = get(f'https://www.basketball-reference.com/friv/standings.fcgi?month=5&day=15&year='+str(year))
    if r.status_code==200:
        soup = BeautifulSoup(r.content, 'html.parser')
        e_table = soup.find('table', attrs={'id': 'standings_e'})
        e_df = pd.read_html(str(e_table))[0]
        w_table = soup.find('table', attrs={'id': 'standings_w'})
        w_df = pd.read_html(str(w_table))[0]
        e_df.rename(columns={'Eastern Conference': 'TEAM','W':str(year)}, inplace=True)
        w_df.rename(columns={'Western Conference': 'TEAM','W':str(year)}, inplace=True)
        e_df['TEAM'] = e_df.TEAM.str.replace(r'([^\s\w])+','')
        w_df['TEAM'] = w_df.TEAM.str.replace(r'([^\s\w])+','')
        e_df = e_df[~e_df.TEAM.str.contains("Division")]
        w_df = w_df[~w_df.TEAM.str.contains("Division")]
        e_df = e_df.iloc[:, :-7]
        w_df = w_df.iloc[:, :-7]
        if east_master.empty is True:
            east_master = e_df.copy()
        else:
             east_master = pd.merge(east_master, e_df, how='outer', on='TEAM')
        if west_master.empty is True:
            west_master = w_df.copy()
        else:
             west_master = pd.merge(west_master, w_df, how='outer',on='TEAM')
        del e_df
        del w_df
        

In [104]:
east = east_master.copy()

In [156]:
#Need to recalibrate for franchise re-locating cities
east['TEAM'] = east.TEAM.str.replace('New Jersey Nets', 'Brooklyn Nets')
east['TEAM'] = east.TEAM.str.replace('Charlotte Bobcats','Charlotte Hornets')
east['TEAM'] = east.TEAM.str.replace('New Orleans Hornets','Charlotte Hornets')


In [169]:
#Since the tables scrapped division titles(strings) everything in those tables are objects so we need to covert to data type
east = east.replace('nan', 0)
east = east.replace(np.NaN, 0)

In [178]:
east = east.set_index('TEAM')

In [183]:
east = east.astype('int64')

In [186]:
#this is to merge the teams movings cities/changing names
east = east.select_dtypes(np.number).max(level=0)

In [196]:
east.head()


,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
TEAM,,,,,,,,,,,,,,,,,,,,,
Miami Heat,52,50,36,25,42,59,52,44,15,43,...,58,46,66,54,37,48,41,44,39,41
New York Knicks,50,48,30,37,39,33,23,33,23,32,...,42,36,54,37,17,32,31,29,17,21
Philadelphia 76ers,49,56,43,48,33,43,38,35,40,41,...,41,35,34,19,18,10,28,52,51,39
Orlando Magic,41,43,44,42,21,36,36,40,52,59,...,52,37,20,23,25,35,29,25,42,30
Boston Celtics,35,36,49,44,36,45,33,24,66,62,...,56,39,41,25,40,48,53,55,49,43
Brooklyn Nets,31,26,52,49,47,42,49,41,34,34,...,24,22,49,44,38,21,20,28,42,30
Washington Wizards,29,19,37,37,25,45,42,41,43,19,...,23,20,29,44,46,41,49,43,32,24
Indiana Pacers,56,41,42,48,61,44,41,35,36,36,...,37,42,49,56,38,45,42,48,48,39
Charlotte Hornets,49,46,44,47,41,18,26,33,32,35,...,34,7,21,43,33,48,36,36,39,23


In [315]:
#subset for 
atlantic = east.query('TEAM ==("New York Knicks","Boston Celtics", "Toronto Raptors", "Philadelphia 76ers", "Brooklyn Nets")')
atldiv_10 = atlantic.T.tail(10).cumsum(0)
atldiv_20 = atlantic.T.cumsum(axis=0)

In [1]:
#update the standings manually
atlantic['2020'] =[21,43,48,35,53]

In [316]:
atldiv_10

TEAM,New York Knicks,Philadelphia 76ers,Boston Celtics,Brooklyn Nets,Toronto Raptors
2011,42,41,56,24,22
2012,78,76,95,46,45
2013,132,110,136,95,79
2014,169,129,161,139,127
2015,186,147,201,177,176
2016,218,157,249,198,232
2017,249,185,302,218,283
2018,278,237,357,246,342
2019,295,288,406,288,400
2020,316,331,454,323,453


In [298]:
import matplotlib.colors
cmap = ListedColormap(['darkorange', 'mediumblue','green', 'k', 'r'])

In [322]:
import bar_chart_race as bcr

bcr.bar_chart_race(df = atldiv_10, title="The Atlantic Divison Cumulative Wins (2010-2020)",
                   orientation='h', cmap=cmap, period_length=1000, fixed_max=True,
                   period_fmt='Season: {x}')